In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import numpy as np

def summarize_df(df, col):
    df_summary = df.groupby('Time').agg(
        Median=(col, 'median'),
        Q1=(col, lambda x: x.quantile(0.25)),
        Q3=(col, lambda x: x.quantile(0.75)),
        D1=(col, lambda x: x.quantile(0.1)),
        D9=(col, lambda x: x.quantile(0.9)),
        Min=(col, 'min'),
        Max=(col, 'max'),
    ).reset_index()
    return df_summary

def plot_visual(df):
    df_basal_insulin = summarize_df(df, 'basal_Insulin Delivered (U)')
    df_bolus_insulin = summarize_df(df, 'bolus_Insulin Delivered (U)')

    df_autoXmanual = df.groupby("Time")["bolus_Trigger"].max()
    colors = np.where(df_autoXmanual.values == 0, 'blue',
                        np.where(df_autoXmanual.values == 1, 'purple',
                        np.where(df_autoXmanual.values == 2, 'red', df_autoXmanual.values)))

    # Initialize the Dash app
    app = Dash(__name__)

    # Create a line plot using Plotly
    def create_plot():
        fig = go.Figure()

        # Add median line
        fig.add_trace(go.Scatter(
            x=df_basal_insulin['Time'],
            y=df_basal_insulin['Median'],
            mode='lines',
            name='Median',
            line=dict(color='green')
        ))

        # Add first quartile (Q1) line
        fig.add_trace(go.Scatter(
            x=df_basal_insulin['Time'],
            y=df_basal_insulin['Q1'],
            mode='lines',
            name='Q1 and Q3',
            line=dict(color='orange', dash='dot'),
            showlegend=False  # Hide legend for this trace
        ))

        # Add third quartile (Q3) line
        fig.add_trace(go.Scatter(
            x=df_basal_insulin['Time'],
            y=df_basal_insulin['Q3'],
            mode='lines',
            line=dict(color='orange', dash='dot'),
            fill='tonexty',  # This fills the area between the Q1 and Q3 lines
            fillcolor='rgba(255, 165, 0, 0.3)',  # Set fill color with transparency
            name='Q1 and Q3'  # Use same label for legend
        ))

            # Segment bolus line based on threshold, but hide individual segments from legend
        for i in range(len(df_bolus_insulin) - 1):
            x_segment = [df_bolus_insulin['Time'].iloc[i], df_bolus_insulin['Time'].iloc[i + 1]]
            y_segment = [df_bolus_insulin['Max'].iloc[i], df_bolus_insulin['Max'].iloc[i + 1]]
            
            # Choose color based on threshold
            #color = 'red' if df_bolus_insulin['Max'].iloc[i] >= threshold else 'blue'
            color = colors[i]
            
            # Add segment as a separate trace, without showing it in the legend
            fig.add_trace(go.Scatter(
                x=x_segment,
                y=y_segment,
                mode='lines+markers',
                yaxis="y2",
                line=dict(color=color, width=2),
                showlegend=False,
                marker=dict(
                    symbol='circle',  # Choose marker shape
                    size=8,           # Set marker size
                    color=color,      # Set marker color
                    opacity=0.8,      # Set marker opacity
                )
            ))

        # Add a "dummy" trace for the legend
        fig.add_trace(go.Scatter(
            x=[None],  # No actual points
            y=[None],
            mode='lines',
            name='Bolus Insulin',
            line=dict(color='blue')  # Use one of the colors to represent the legend item
        ))

        # Update layout
        fig.update_layout(
            title='Summary Statistics by Time of Day',
            xaxis_title='Time of Day',
            yaxis_title='Basal',
            yaxis2=dict(
                title='Bolus',
                overlaying='y',
                side='right',
                showgrid=False  # Hide grid for secondary y-axis if desired
            ),
            xaxis=dict(tickmode='array'),
            legend=dict(x=0.01, y=0.99)
        )

        return fig

    # Define the layout of the app
    app.layout = html.Div([
        html.H1("Time Series Summary Statistics"),
        dcc.Graph(figure=create_plot())  # Use the create_plot function to generate the figure
    ])

    # Run the app
    if __name__ == '__main__':
        app.run_server(debug=True)
